In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import os
import logging

from jax import random
import wandb

from src.models import make_PoN_Ens_loss as make_loss
import src.data
from src.data import NumpyLoader
from src.utils.training import setup_training, train_loop
from experiments.configs.toy_pon_ens import get_config

In [11]:
os.environ['XLA_FLAGS'] = "--xla_gpu_force_compilation_parallelism=1"

In [12]:
os.environ['WANDB_NOTEBOOK_NAME'] = 'train_pon_ens.ipynb'
# ^ W&B doesn't know how to handle VS Code notebooks.

wandb.login()

True

In [13]:
rng = random.PRNGKey(0)

In [14]:
config = get_config()

In [15]:
data_gen_fn = getattr(src.data, config.dataset_name)
train_dataset, test_dataset, val_dataset = data_gen_fn(**config.dataset.to_dict())
train_loader = NumpyLoader(train_dataset, config.batch_size)
val_loader = NumpyLoader(val_dataset, config.batch_size)
test_loader = NumpyLoader(test_dataset, config.batch_size)

In [16]:
setup_rng, rng = random.split(rng)
init_x = train_dataset[0][0]
init_y = train_dataset[0][1]

model, state = setup_training(config, setup_rng, init_x, init_y)

+---------------------------------------------+------------+--------+-----------+--------+
| Name                                        | Shape      | Size   | Mean      | Std    |
+---------------------------------------------+------------+--------+-----------+--------+
| batch_stats/nets_0/layer_0/BatchNorm_0/mean | (100,)     | 100    | 0.0       | 0.0    |
| batch_stats/nets_0/layer_0/BatchNorm_0/var  | (100,)     | 100    | 1.0       | 0.0    |
| batch_stats/nets_0/layer_1/BatchNorm_0/mean | (100,)     | 100    | 0.0       | 0.0    |
| batch_stats/nets_0/layer_1/BatchNorm_0/var  | (100,)     | 100    | 1.0       | 0.0    |
| batch_stats/nets_1/layer_0/BatchNorm_0/mean | (100,)     | 100    | 0.0       | 0.0    |
| batch_stats/nets_1/layer_0/BatchNorm_0/var  | (100,)     | 100    | 1.0       | 0.0    |
| batch_stats/nets_1/layer_1/BatchNorm_0/mean | (100,)     | 100    | 0.0       | 0.0    |
| batch_stats/nets_1/layer_1/BatchNorm_0/var  | (100,)     | 100    | 1.0       | 0.0    |

In [17]:
state = train_loop(
    model, state, config, rng, make_loss, make_loss, train_loader, val_loader,
    # test_loader,
    wandb_kwargs={
        'mode': 'online',
        # 'notes': '',
    },
)

2022-10-07 14:44:23.337920: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-07 14:44:23.865626: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.8/lib64
2022-10-07 14:44:23.865694: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.8/lib64
2022-10-07 14:44:23.865701: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned abo

  0%|          | 0/50 [00:00<?, ?it/s]

epoch:   1 - train loss: 1.45768, val loss: 1.38024, train err: 1.0775, val err: 0.9226, lr: 0.00010
Best val_err
epoch:   2 - train loss: 1.37613, val loss: 1.35193, train err: 0.9144, val err: 0.8660, lr: 0.00010
Best val_err
epoch:   3 - train loss: 1.30279, val loss: 1.32709, train err: 0.7677, val err: 0.8164, lr: 0.00010
Best val_err
epoch:   4 - train loss: 1.24579, val loss: 1.30444, train err: 0.6539, val err: 0.7714, lr: 0.00010
Best val_err
epoch:   5 - train loss: 1.19725, val loss: 1.28225, train err: 0.5576, val err: 0.7276, lr: 0.00010
Best val_err
epoch:   6 - train loss: 1.16232, val loss: 1.26001, train err: 0.4890, val err: 0.6841, lr: 0.00010
Best val_err
epoch:   7 - train loss: 1.13692, val loss: 1.23529, train err: 0.4399, val err: 0.6361, lr: 0.00010
Best val_err
epoch:   8 - train loss: 1.12147, val loss: 1.20826, train err: 0.4112, val err: 0.5840, lr: 0.00010
Best val_err
epoch:   9 - train loss: 1.10881, val loss: 1.18108, train err: 0.3885, val err: 0.5322,

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/err,█▇▆▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▇▆▅▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
val/err,██▇▇▆▆▅▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/loss,██▇▇▆▆▆▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
best_epoch,50
best_val_err,0.1175
epoch,50
learning_rate,0.0001
train/err,0.13362
